In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
from selenium import webdriver

# Getting the transcripts

### Figuring out the logic

In [36]:
browser.get("https://fangj.github.io/friends/")

In [73]:
link1=browser.find_element(by='xpath',value="/html/body/ul[1]/li[1]/a")
link1.get_attribute('href')

'https://fangj.github.io/friends/season/0101.html'

In [40]:
webdriver.ActionChains(browser).click(link1).perform()

In [43]:
raw_html = browser.page_source

In [44]:
processed_html = BeautifulSoup(raw_html,'html.parser')

In [53]:
text = processed_html.text.split("[")[1:]

In [57]:
" ".join(text).strip()

'Scene: Central Perk, Chandler, Joey, Phoebe, and Monica are there.]\nMonica: There\'s nothing to tell! He\'s just some guy\nI work with!\nJoey: C\'mon, you\'re going out with the guy! There\'s\ngotta be something wrong with him!\nChandler: All right Joey, be\nnice.\xa0 So does he have a hump? A hump and a hairpiece?\nPhoebe: Wait, does he eat chalk?\n(They all stare, bemused.)\nPhoebe: Just, \'cause, I don\'t want her to go through\nwhat I went through with Carl- oh!\nMonica: Okay, everybody relax. This is not even a\ndate. It\'s just two people going out to dinner and- not having sex.\nChandler: Sounds like a date to me.\n Time Lapse]\nChandler: Alright, so I\'m back in high school, I\'m\nstanding in the middle of the cafeteria, and I realize I am totally naked.\nAll: Oh, yeah. Had that dream.\nChandler: Then I look down, and I realize there\'s a\nphone... there.\nJoey: Instead of...?\nChandler: That\'s right.\nJoey: Never had that dream.\nPhoebe: No.\nChandler: All of a sudden, the 

### Getting all the transcripts

In [9]:
browser = webdriver.Edge()
browser.get("https://fangj.github.io/friends/")

In [10]:
transcript = pd.DataFrame(columns=['Season','Episode','Transcript'])
transcript.index

Index([], dtype='object')

In [11]:
for i in range(1,11):
    for j in range(1,26):
        try:
            link = browser.find_element(by='xpath', value=f"/html/body/ul[{i}]/li[{j}]/a")
        except:
            continue
        browser2 = webdriver.Edge()
        browser2.get(link.get_attribute('href'))
        html = BeautifulSoup(browser2.page_source)
        text = html.text.split("[")[1:]
        final_text = " ".join(text).strip()
        transcript.loc[len(transcript.index)] = [i,j,final_text]
        browser2.close()

In [52]:
#Season 2 episode 3 doesnt have a '[' in the transcript, so it had to be scraped manually.
browser.get('https://fangj.github.io/friends/season/0203.html')
html = BeautifulSoup(browser.page_source)

In [38]:
transcript.iloc[26] = pd.Series({'Season': 2,'Episode':3,'Transcript': html.text[199:]})

In [39]:
transcript.iloc[26]

Season                                                        2
Episode                                                       3
Transcript    CHANDLER: Hey.\nMONICA: So how was Joan?\nCHAN...
Name: 26, dtype: object

In [41]:
#s07e24 is a special/extra episode with no rating.
transcript.drop(164,axis=0,inplace=True)
transcript = transcript.reset_index().drop('index',axis=1)

In [40]:
transcript.to_csv("transcripts.csv",index=False)

### Getting the IMDb ratings

In [4]:
browser = webdriver.Edge()

In [5]:
import fnmatch
ratings = pd.DataFrame(columns=['Season','Episode','Rating'])
for i in range(1,11):
    url = f'https://www.imdb.com/title/tt0108778/episodes?season={i}'
    browser.get(url)
    parsed_html = BeautifulSoup(browser.page_source)
    info = browser.find_element(by='class name',value="list.detail.eplist").text.split("\n")
    season = pd.DataFrame(columns = ['Episode', 'Rating'])
    season['Episode'] = fnmatch.filter(info, f"*S{i},*")
    season['Rating'] = fnmatch.filter(info, "*(*)")
    splitter = lambda x : x.split(",")[0]
    season['Season'] = season['Episode'].apply(splitter)
    splitter = lambda x : x.split(",")[1]
    season['Episode'] = season['Episode'].apply(splitter)
    season = pd.DataFrame({'Season': season.Season, 'Episode':season.Episode,'Rating':season.Rating})
    ratings = ratings.append(season)
browser.close()

In [181]:
ratings.to_csv('ratings.csv',index=False)

In [40]:
transcripts = pd.read_csv('transcripts.csv')

In [16]:
ratings = pd.read_csv('ratings.csv')

In [17]:
transcripts, ratings

(     Season  Episode                                         Transcript
 0         1        1  Scene: Central Perk, Chandler, Joey, Phoebe, a...
 1         1        2  Scene Central Perk, everyone's there.]\nMonica...
 2         1        3  Scene: Central Perk, everyone but Phoebe is th...
 3         1        4  Scene: Central Perk, everyone is there except ...
 4         1        5  Scene: Central Perk, all six are there.]\nMoni...
 ..      ...      ...                                                ...
 223      10       13  Scene: Central Perk. Everyone's sitting on the...
 224      10       14  Scene: Chandler and Monica's apartment. They a...
 225      10       15  Flashback scene from last week, Monica and Cha...
 226      10       16  Scene: Joey's place. Rachel and Joey are talki...
 227      10       17  Scene: Monica and Chandler's apartment. It's a...
 
 [228 rows x 3 columns],
     Season Episode        Rating
 0       S1     Ep1   8.2 (8,746)
 1       S1     Ep2   7.9 (6,

In [18]:
#Dealing with merging ratings of episodes with two parts
ratings.iloc[35] = {'Season': 'S2', 'Episode': 'Ep12_13', 'Rating': '8.6 (5,192)'}
ratings.iloc[95] = {'Season': 'S4', 'Episode': 'Ep23_24', 'Rating': '8.9 (5,169)'}
ratings.iloc[119] = {'Season': 'S5', 'Episode': 'Ep23_24', 'Rating': '8.9 (4,805)'}
ratings.iloc[135] = {'Season': 'S6', 'Episode': 'Ep15_16', 'Rating': '8.4 (4,221)'}
ratings.iloc[144] = {'Season': 'S6', 'Episode': 'Ep24_25', 'Rating': '8.9 (5,001)'}
ratings.iloc[168] = {'Season': 'S7', 'Episode': 'Ep23_24', 'Rating': '9.0 (4,700)'}
ratings.iloc[192] = {'Season': 'S8', 'Episode': 'Ep23_24', 'Rating': '8.8 (4,207)'}
ratings.iloc[216] = {'Season': 'S9', 'Episode': 'Ep23_24', 'Rating': '8.5 (3,924)'}

In [19]:
splitter = lambda x: x.split("S")[1]
ratings.Season = ratings.Season.apply(splitter)

In [20]:
splitter = lambda x: x.split("Ep")[1]
ratings.Episode = ratings.Episode.apply(splitter)

In [21]:
ratings.drop([36,96,120,136,145,169,193,217],axis=0,inplace=True) #remove duplicate ratings after merging

In [22]:
ratings #ratings now has same rows as transcripts

,Season,Episode,Rating
0,1,1,"8.2 (8,746)"
1,1,2,"7.9 (6,745)"
2,1,3,"8.0 (6,346)"
3,1,4,"8.0 (6,162)"
4,1,5,"8.4 (6,153)"
...,...,...,...
230,10,13,"8.4 (4,225)"
231,10,14,"8.5 (4,103)"
232,10,15,"8.4 (3,857)"
233,10,16,"8.8 (4,466)"


In [23]:
ratings = ratings.reset_index().drop('index',axis=1)

In [25]:
ratings.to_csv('final_ratings.csv',index=False)

In [43]:
final_data = transcripts.join(ratings,rsuffix='a').drop(['Seasona','Episodea'],axis=1)

In [44]:
final_data

,Season,Episode,Transcript,Rating
0,1,1,"Scene: Central Perk, Chandler, Joey, Phoebe, a...","8.2 (8,746)"
1,1,2,"Scene Central Perk, everyone's there.]\nMonica...","7.9 (6,745)"
2,1,3,"Scene: Central Perk, everyone but Phoebe is th...","8.0 (6,346)"
3,1,4,"Scene: Central Perk, everyone is there except ...","8.0 (6,162)"
4,1,5,"Scene: Central Perk, all six are there.]\nMoni...","8.4 (6,153)"
...,...,...,...,...
222,10,13,Scene: Central Perk. Everyone's sitting on the...,"8.4 (4,225)"
223,10,14,Scene: Chandler and Monica's apartment. They a...,"8.5 (4,103)"
224,10,15,"Flashback scene from last week, Monica and Cha...","8.4 (3,857)"
225,10,16,Scene: Joey's place. Rachel and Joey are talki...,"8.8 (4,466)"


In [46]:
final_data.to_csv('final_data.csv',index=False)